<a href="https://colab.research.google.com/github/ravgupta11/Machine-Learning-Deep-Learning-Matrix/blob/main/TextBoundingBoxDetectron2_Synth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://thor.robots.ox.ac.uk/~vgg/data/scenetext/SynthText.zip

--2021-08-13 05:08:29--  https://thor.robots.ox.ac.uk/~vgg/data/scenetext/SynthText.zip
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40717671794 (38G) [application/zip]
Saving to: ‘SynthText.zip’

SynthText.zip        98%[==================> ]  37.28G  31.8MB/s    eta 21s    

In [ ]:
!unzip SynthText.zip; rm SynthText.zip;

In [ ]:
#!pip install pyyaml==5.1
# This is the current pytorch version on Colab. Uncomment this if Colab changes its pytorch version
# !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

1.9.0+cu102 True


In [ ]:
#importing detectron2 libraries
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode


In [ ]:
import numpy as np
import pandas as pd
import itertools
import os, json, cv2, random
from google.colab.patches import cv2_imshow #for colab only
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
torch.cuda.empty_cache()

In [ ]:
#TO CREATE COCO ANNOTATIONS
#import numpy as np
#import os
#import cv2
#import random
#import scipy.io
#from detectron2.data.datasets.coco import convert_to_coco_json

#mat = scipy.io.loadmat('/content/SynthText/gt.mat')

#dataset = []
#for image_id in range(len(mat['imnames'][0])):
  #record = {}
  #record['image_id'] = image_id
  #record['file_name'] = os.path.join(os.getcwd(),'SynthText', mat['imnames'][0][image_id][0])
  #record['width'], record['height'] = cv2.imread(record['file_name']).shape[:2]
  #record['annotations'] = []
  #objs = mat['wordBB'][0][image_id]
  #if len(objs.shape) < 3:
  #  continue
  #for object_id in range(objs.shape[-1]):
  #  xmax, ymax = np.max(objs[:,:,object_id], axis = 1)
  #  xmin, ymin = np.min(objs[:,:, object_id], axis = 1)
  #  obj = { 'bbox': [xmin, ymin, xmax, ymax],
  #         "bbox_mode": BoxMode.XYXY_ABS,
  #         'category_id' : 0
  #         }

  #  record['annotations'].append(obj)
  #dataset.append(record)

#random.shuffle(dataset)

#def get_dataset(dataset, _set='train'):
#    if _set == 'train' :
#      return dataset[:int(0.8 * len(dataset))]
#    return dataset[int(0.8 * len(dataset)): ]

#for d in ["train", "val"]:
#    DatasetCatalog.register("my_dataset_" + d, lambda d=d: get_dataset(dataset, d))
#   MetadataCatalog.get("my_dataset_" + d).set(thing_classes=['text'])

#convert_to_coco_json('my_dataset_train', output_file = '/content/train.json', allow_cached=False)
#convert_to_coco_json('my_dataset_val', output_file = '/content/val.json', allow_cached = False)
#!cp '/content/train.json' '/content/drive/MyDrive/SynthTextTrain.json';cp '/content/val.json' '/content/drive/MyDrive/SynthTextVal.json'

In [ ]:
!cp '/content/drive/MyDrive/SynthTextTrain.json' '/content/train.json';cp '/content/drive/MyDrive/SynthTextVal.json' '/content/val.json'
!cp '/content/drive/MyDrive/detectron2_textdetection_synthtext_50k.pth' '/content/pretrained_weights.pth'

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("train_dataset", {},"/content/train.json", "")
register_coco_instances("val_dataset", {}, "/content/val.json", "")

In [ ]:
logo_metadata = MetadataCatalog.get('train_dataset')
dataset_dicts = DatasetCatalog.get('train_dataset')
print(len(dataset_dicts))
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=logo_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    #cv2.imshow('frame', out.get_image()[:, :, ::-1])
    #cv2.waitKey(3000)
    #cv2.destroyAllWindows()
    cv2_imshow(out.get_image()[:, :, ::-1])

[08/11 08:35:41 d2.data.datasets.coco]: Loading /content/train.json takes 36.06 seconds.
WARNING [08/11 08:35:41 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[08/11 08:35:46 d2.data.datasets.coco]: Loaded 667473 images in COCO format from /content/train.json


In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator


class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("logo_val", exist_ok=True)
        output_folder = "logo_val"

    return COCOEvaluator(dataset_name, ('bbox',), True, output_folder)

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = '/content/pretrained_weights.pth'

cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 4



cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.50 #original 0.5
cfg.MODEL.RPN.POSITIVE_FRACTION = 0.50 #original 0.5
cfg.MODEL.RPN.NMS_THRESH = 0.7 #original 0.7
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

#cfg.MODEL.RPN.PRE_NMS_TOPK_TRAIN = 150000 #original 12000
#cfg.MODEL.RPN.PRE_NMS_TOPK_TEST = 150000 #original 6000
cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 10000 #original 2000
cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 10000 #original 1000
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16, 32, 64, 128, 256, 512]] # added 16 here 
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.5, 1.0, 2.0, 7.0]] #added 7 here 


cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1


cfg.SOLVER.WARMUP_ITERS = 40
cfg.SOLVER.MAX_ITER = 100000 
cfg.SOLVER.GAMMA = 0.05
cfg.SOLVER.IMS_PER_BATCH = 3
cfg.SOLVER.BASE_LR = 1e-3

cfg.TEST.DETECTIONS_PER_IMAGE = 99000 #original 100 
cfg.TEST.EVAL_PERIOD = 50000

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()
!cp '/content/output/model_final.pth' '/content/drive/MyDrive/detectron_text_detection_synthtext_150k.pth'